In [ ]:
from models import FixedLengthCRNN, VariableLengthCRNN
from dataset import DigitsDataset
from torch.utils.data import DataLoader
from torch import optim
import torch.nn as nn
num_epochs = 100   


# Model training

In [ ]:

# Initialize dataset and data loader
dataset = DigitsDataset('/path/to/your/dataset')
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = FixedLengthCRNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    for images, labels in data_loader:
        optimizer.zero_grad()
        
        outputs = model(images)
        
        # Since outputs are in shape (batch_size, 4, 36) and labels are (batch_size, 4),
        # you need to calculate the loss for each digit and sum up.
        loss = sum([loss_function(outputs[:, i], labels[:, i]) for i in range(4)])
        
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
